# Lab 3 — dimensionality reduction

In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np

from operator import itemgetter

%matplotlib inline
plt.style.use("ggplot")

In [2]:
!hdfs dfs -cat /ix/ml-20m/genome-tags.txt | tail -n 5

{"tagId": 1124, "tag": "writing"}
{"tagId": 1125, "tag": "wuxia"}
{"tagId": 1126, "tag": "wwii"}
{"tagId": 1127, "tag": "zombie"}
{"tagId": 1128, "tag": "zombies"}


In [3]:
!hdfs dfs -cat /ix/ml-20m/genome-scores.txt | tail -n 5

{"relevance": 0.58775, "tagId": 1124, "movieId": 131170}
{"relevance": 0.010749999999999982, "tagId": 1125, "movieId": 131170}
{"relevance": 0.015749999999999986, "tagId": 1126, "movieId": 131170}
{"relevance": 0.11449999999999999, "tagId": 1127, "movieId": 131170}
{"relevance": 0.02174999999999999, "tagId": 1128, "movieId": 131170}


In [4]:
data = sc.textFile("/ix/ml-20m/genome-tags.txt").map(json.loads)
tag2name = dict(data.map(itemgetter("tagId", "tag")).collect())
tagIds = data.map(lambda x: int(x["tagId"]))
ntags = tagIds.max()

In [19]:
dataS = sc.textFile("/ix/ml-20m/genome-scores.txt").map(json.loads)
movieIds = dataS.map(lambda x: int(x["movieId"])).distinct()
nmovies = movieIds.count()

In [20]:
movieIds.take(10)
nmovies

10381

In [ ]:
dM = np.empty((ntags, nmovies,))

for l in dataS.collect():
    dM[l["tagId"] - 1, l["movieId"] - 1] = l["relevance"]

In [ ]:
dM[0,0]